In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd
import math
import keras
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler,MinMaxScaler,RobustScaler
from sklearn.utils import class_weight
from keras import optimizers
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
from keras.models import load_model
import joblib
# import py_vollib.black_scholes.implied_volatility
import multiprocessing
from multiprocessing import Pool
import os
import py_vollib.black_scholes as vb
import statistics
%matplotlib inline
# from tensorflow import set_random_seed
# set_random_seed(9)
import datetime

Using TensorFlow backend.


In [2]:
def c_int(lst):
    df = pd.DataFrame(lst)
    df = df.sort_values(by = 0).reset_index(drop=True)
    cp1 = df[np.logical_and(df[0] >= -2,df[0] <= 2)]
    return (len(cp1)/len(df))*100

In [3]:
# Path settings


path_var = 1

if (path_var == 1):
    filepath = '/home/docboy/Desktop/Option_Pricing/datasets/'
    modelpath = '/home/docboy/Desktop/Option_Pricing/pickles/'


else:
    filepath = '/home/sharan/Desktop/OptionPricing-master/datasets/'
    modelpath = '/home/sharan/Desktop/OptionPricing-master/pickles/'


# nfty_ecdf = filepath + 'REL_feature_set_ECDF_STEP_2_NOSCALED.csv'

# nfty_ecdf = filepath + 'NIFTY50_feature_set_ECDF_complete.csv'
# nfty_ecdf = filepath + 'BANKNIFTY_feature_set_ECDF_complete.csv'
nfty_ecdf = filepath + 'NIFTY50_2019_feature_set_ECDF_complete.csv'

In [4]:
# Set the automated variables + Plus drop all useless columns approach specific


###########################
###############################################

approach = 1                      #------------------>>> # Change value as needed

###############################################
###########################


if (approach == 1):
    feats = pd.read_csv(nfty_ecdf)

feats['date'] = pd.to_datetime(feats['date'])


In [5]:
feats.columns

Index(['date', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10',
       'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'ttm',
       'atmError', 'strike', 'spot', 'opPrice', 'percentage', 'yield03',
       'Target'],
      dtype='object')

In [6]:
test=feats[(feats['date']>=datetime.datetime(2012,8,1))]
test

,date,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f18,f19,ttm,atmError,strike,spot,opPrice,percentage,yield03,Target
0,2019-04-02,-0.009000,-0.005584,-0.003332,-0.002064,-0.000985,0.000137,0.000470,0.003089,0.003577,...,0.011852,0.011949,23.0,103.656637,11300.0,11713.2,512.10,4.531858,6.18,45
1,2019-04-02,-0.009000,-0.005584,-0.003332,-0.002064,-0.000985,0.000137,0.000470,0.003089,0.003577,...,0.011852,0.011949,9.0,100.542489,11650.0,11713.2,153.20,1.315021,6.18,13
2,2019-04-02,-0.009000,-0.005584,-0.003332,-0.002064,-0.000985,0.000137,0.000470,0.003089,0.003577,...,0.011852,0.011949,9.0,98.430252,11900.0,11713.2,40.25,0.338235,6.18,3
3,2019-04-02,-0.009000,-0.005584,-0.003332,-0.002064,-0.000985,0.000137,0.000470,0.003089,0.003577,...,0.011852,0.011949,9.0,97.610000,12000.0,11713.2,21.50,0.179167,6.18,1
4,2019-04-02,-0.009000,-0.005584,-0.003332,-0.002064,-0.000985,0.000137,0.000470,0.003089,0.003577,...,0.011852,0.011949,9.0,101.412987,11550.0,11713.2,192.60,1.667532,6.18,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7664,2020-04-15,-0.139038,-0.079174,-0.057168,-0.044768,-0.040827,-0.025364,-0.024547,-0.020812,-0.013040,...,0.064145,0.084003,8.0,102.589655,8700.0,8925.3,376.75,4.330460,4.08,43
7665,2020-04-15,-0.139038,-0.079174,-0.057168,-0.044768,-0.040827,-0.025364,-0.024547,-0.020812,-0.013040,...,0.064145,0.084003,8.0,101.423864,8800.0,8925.3,311.10,3.535227,4.08,35
7666,2020-04-15,-0.139038,-0.079174,-0.057168,-0.044768,-0.040827,-0.025364,-0.024547,-0.020812,-0.013040,...,0.064145,0.084003,8.0,100.284270,8900.0,8925.3,255.90,2.875281,4.08,28
7667,2020-04-15,-0.139038,-0.079174,-0.057168,-0.044768,-0.040827,-0.025364,-0.024547,-0.020812,-0.013040,...,0.064145,0.084003,8.0,99.170000,9000.0,8925.3,205.05,2.278333,4.08,22


In [7]:
log_return = test.loc[:,['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10',
       'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19']]
t = 1/250
var = np.var(log_return,axis=1)/t
h_sigma = np.sqrt(var)
test['h_vola'] = h_sigma
test.reset_index(inplace = True, drop = True)

In [8]:
test

,date,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f19,ttm,atmError,strike,spot,opPrice,percentage,yield03,Target,h_vola
0,2019-04-02,-0.009000,-0.005584,-0.003332,-0.002064,-0.000985,0.000137,0.000470,0.003089,0.003577,...,0.011949,23.0,103.656637,11300.0,11713.2,512.10,4.531858,6.18,45,0.09191
1,2019-04-02,-0.009000,-0.005584,-0.003332,-0.002064,-0.000985,0.000137,0.000470,0.003089,0.003577,...,0.011949,9.0,100.542489,11650.0,11713.2,153.20,1.315021,6.18,13,0.09191
2,2019-04-02,-0.009000,-0.005584,-0.003332,-0.002064,-0.000985,0.000137,0.000470,0.003089,0.003577,...,0.011949,9.0,98.430252,11900.0,11713.2,40.25,0.338235,6.18,3,0.09191
3,2019-04-02,-0.009000,-0.005584,-0.003332,-0.002064,-0.000985,0.000137,0.000470,0.003089,0.003577,...,0.011949,9.0,97.610000,12000.0,11713.2,21.50,0.179167,6.18,1,0.09191
4,2019-04-02,-0.009000,-0.005584,-0.003332,-0.002064,-0.000985,0.000137,0.000470,0.003089,0.003577,...,0.011949,9.0,101.412987,11550.0,11713.2,192.60,1.667532,6.18,16,0.09191
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7664,2020-04-15,-0.139038,-0.079174,-0.057168,-0.044768,-0.040827,-0.025364,-0.024547,-0.020812,-0.013040,...,0.084003,8.0,102.589655,8700.0,8925.3,376.75,4.330460,4.08,43,0.83796
7665,2020-04-15,-0.139038,-0.079174,-0.057168,-0.044768,-0.040827,-0.025364,-0.024547,-0.020812,-0.013040,...,0.084003,8.0,101.423864,8800.0,8925.3,311.10,3.535227,4.08,35,0.83796
7666,2020-04-15,-0.139038,-0.079174,-0.057168,-0.044768,-0.040827,-0.025364,-0.024547,-0.020812,-0.013040,...,0.084003,8.0,100.284270,8900.0,8925.3,255.90,2.875281,4.08,28,0.83796
7667,2020-04-15,-0.139038,-0.079174,-0.057168,-0.044768,-0.040827,-0.025364,-0.024547,-0.020812,-0.013040,...,0.084003,8.0,99.170000,9000.0,8925.3,205.05,2.278333,4.08,22,0.83796


In [9]:
maxVal=5
minVal=0
bins=50
bin_width=step=(maxVal-minVal)/bins
thresh=[]
for i in range(0,bins):
    val=(minVal+(i*step))
    thresh.append(round(val,1))
    
print(thresh)

def findBin(val):
    for t in range(1,len(thresh)):
        if(val < thresh[t]):
            return (t-1)
        elif(val >= thresh[-1]):
            return round(thresh[-1]*10)

[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3.0, 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9, 4.0, 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4.8, 4.9]


In [10]:
BSM=[]
for i,row in test.iterrows():
#     test.loc[i,'date']
    BSMprice=vb.black_scholes('c',test.loc[i,'spot'], 
               test.loc[i,'strike'],test.loc[i,'ttm']/365,
               test.loc[i,'yield03']/100,
              test.loc[i,'h_vola'])
#     print(BSMprice)
#     print(test.loc[i,'opPrice'])
    BSMprice_norm = BSMprice/test.loc[i,'strike'] *100
#     print(BSMprice_norm)
    BSMBin=findBin(BSMprice_norm)
#     print(BSMBin)
    BSM.append(BSMBin)
#     break

In [11]:
CP=[]
dist_sum = 0
for i in range(len(test['Target'])):
    dist = abs(test.loc[i,'Target'] - BSM[i])
    CP.append(dist)
#     print(dist)
    dist_sum = dist_sum + dist
#     break

print("NEM:{}".format((dist_sum*bin_width)/len(test['Target'])))

NEM:0.311487808058417


In [12]:
print("Confidence Interval:{}".format(c_int(CP)))

Confidence Interval:55.80910157778067


In [13]:
blen=len(test['Target'])
blen

7669